In [ ]:
# ----------------------------
# Script: roc_curve_analysis.py
# Purpose: Perform ROC curve analysis on selected lncRNAs
# Author: [Your Name]
# ----------------------------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------
# Step 1: Load Data
# ----------------------------

# Load expression matrix with selected features
X = pd.read_csv("data/selected_lncRNA_expression.csv", index_col=0)

# Load labels (0 = normal, 1 = tumor)
labels = pd.read_csv("data/sample_labels.csv", index_col=0)
y = labels['label'].values

# Ensure alignment
if X.shape[0] != len(y):
    X = X.T  # If needed

# ----------------------------
# Step 2: Train-Test Split
# ----------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# ----------------------------
# Step 3: Train Model (Logistic Regression)
# ----------------------------

clf = LogisticRegression(max_iter=1000, solver='liblinear')
clf.fit(X_train, y_train)
y_pred_prob = clf.predict_proba(X_test)[:, 1]

# ----------------------------
# Step 4: ROC Curve and AUC
# ----------------------------

fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# ----------------------------
# Step 5: Plot ROC Curve
# ----------------------------

plt.figure(figsize=(8, 6))
sns.set(style="whitegrid")

plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Analysis of Selected lncRNAs')
plt.legend(loc="lower right")

plt.tight_layout()
plt.savefig("results/roc_curve_selected_lncRNAs.png")
plt.close()

print(f"AUC Score: {roc_auc:.4f}")
